In [1]:
import openai
import os

In [ ]:
import google.auth
from googleapiclient.discovery import build
import googleapiclient.errors
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

could not get google to work, use this instead: "https://www.geeksforgeeks.org/python-downloading-captions-from-youtube/"

In [13]:
from youtube_transcript_api import YouTubeTranscriptApi


### Youtube API DOES NOT WORK

In [2]:
# Set up OpenAI API credentials
with open("alexytapi.txt") as f:
    yt_api_key = f.read().strip()


In [40]:
# Load the client ID and secret from the JSON file
creds = Credentials.from_authorized_user_file('alexoauth2.json', scopes=['https://www.googleapis.com/auth/youtube.force-ssl'])

# If the credentials have expired, refresh them
if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())

In [6]:
# Set up the OAuth2 flow
scopes = ['https://www.googleapis.com/auth/youtube.force-ssl']
flow = InstalledAppFlow.from_client_secrets_file(
    'alex3.json', scopes=scopes)

# Run the flow to get an access token
creds = flow.run_local_server(port=0)

# Save the credentials to a file
with open('creds.json', 'w') as f:
    f.write(creds.to_json())
    
# Load the credentials from the file
creds = Credentials.from_authorized_user_file('creds.json', scopes=scopes)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=398351462937-1scg1d6pfqn4ugcrg6pt91qpak0bcr51.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58441%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=zR2cg417F5RCUoZRSdmVelolkl78gR&access_type=offline


In [7]:
youtube = build('youtube', 'v3', credentials=creds)

In [9]:
def download_transcript(video_id):
    #youtube = build('youtube', 'v3', developerKey=api_key)
    
    # Retrieve the video caption
    captions = youtube.captions().list(
        part='id', videoId=video_id
    ).execute()
    
    # Extract the caption track ID
    caption_id = captions['items'][0]['id']
    
    # Download the caption track
    caption = youtube.captions().download(
        id=caption_id,
        tfmt='srt'
    ).execute()
    
    return caption

In [9]:
video_id = "UF8uR6Z6KLc"
#transcript = download_transcript(video_id)

In [10]:
captions = youtube.captions().list(
        part='id', videoId=video_id
    ).execute()

# Extract the caption track ID
caption_id = captions['items'][0]['id']

# Download the caption track
caption = youtube.captions().download(
        id=caption_id,
        tfmt='vtt'
    ).execute()

HttpError: <HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/captions/AUieDabFVqU1ci7dx-cP4UHlWd3o9cjfS3ln-9GWe93N?tfmt=vtt returned "The permissions associated with the request are not sufficient to download the caption track. The request might not be properly authorized, or the video order might not have enabled third-party contributions for this caption.". Details: "[{'message': 'The permissions associated with the request are not sufficient to download the caption track. The request might not be properly authorized, or the video order might not have enabled third-party contributions for this caption.', 'domain': 'youtube.caption', 'reason': 'forbidden', 'location': 'id', 'locationType': 'parameter'}]">

In [12]:
try:
    caption = youtube.captions().download(
        id=caption_id,
        tfmt='srt'
    ).execute()
except googleapiclient.errors.Error as e:
    print('An HTTP error %d occurred:\n%s' % (e.resp.status, e.content))

TypeError: Got an unexpected keyword argument tfmt

### Youtube Workaround

In [17]:
transcript_raw = YouTubeTranscriptApi.get_transcript("UF8uR6Z6KLc")

In [18]:
transcript = ""
for t in transcript_raw:
    transcript += t["text"]

In [25]:
transcript = transcript[0:(len(transcript)//2)]

### OpenAI

In [26]:
# Set up OpenAI API credentials
with open("keys/alexapi.txt") as f:
    api_key = f.read().strip()

openai.api_key = api_key

In [27]:
# Define the prompt to generate text from
prompt = "summarize in 300 words: " + transcript

# Generate text using the ChatGPT model
response = openai.Completion.create(
  engine="curie",
  prompt=prompt,
  max_tokens=200,
  temperature=0.5
)


In [28]:
# Print the generated text
print(response["choices"][0]["text"])

s gone,and I was free.I had a blank check to follow my heart.I was free to do anything.I was free to drop out and trust that I would be OK.I had nothing to lose.I had only just begun to live.I had so much to learn, so much to experience.I could go anywhere, do anything, be anyone.I could meet anyone, do anything, be anyone.I could travel any distance, be anyone.I could speak in public, be anyone.I could write about anything, be anyone.I could be anyone I wanted to be.For the first time in my life, I felt truly free.And that freedom is what has made all thedifference.So the next time you feel like your world is caving in,remember that your world was never meant to be on a steady foundation.It is meant to be in the wind.And so I am going to leave you with a story.This is the


In [29]:
transcript

'This program is brought to you by Stanford University.Please visit us at stanford.eduThank You. I am honored to be with you today at your commencementfrom one of the finest universities in the world.Truth be told I never graduated from collegeand this is the closest I\'ve ever gotten to a college graduation.Today I want to tell you three stories from my life. That\'s it.No big deal. Just three stories.The first story is about connecting the dots.I dropped out of Reed College after the first 6 months,but then stayed around as a drop-infor another 18 months or so before I really quit.So why did I drop out?It started before I was born.My biological mother was a young, unwed graduate student,and she decided to put me up for adoption.She felt very strongly that I should be adopted by college graduates,so everything was all set for me tobe adopted at birth by a lawyer and his wife.Except that when I popped out they decidedat the last minute that they really wanted a girl.So my parents, who 

In [ ]:
# look into coding to reduce input
# remove useless words, or partition it